In [1]:
// we need to specify the path and explicitly load the CAF library
#pragma cling add_library_path("/usr/local/lib")
#pragma cling add_include_path("/usr/local/include")
#pragma cling load("/usr/local/lib/libcaf_core.so")

# Request with then

This example demonstrates the usage of **then** when you use **request**. Usage of **then** keeps the mailbox of an actor active i.e. messages to its other behaviors can still be sent.

In [2]:
#include <caf/all.hpp>

## An actor that takes long time to respond

In [4]:
// This is an event based actor that does something that takes
// long time before it returns its result
caf::behavior takes_long_time_to_respond(caf::event_based_actor *self) {
  return {

      // behaviors of our actor

      // only behavior its implements is a function
      // that takes nothing but returns a value.
      // However, it does computation heavy stuff (simulated by sleeping) before
      // returning its result.
      [=](int x) -> int {
        std::this_thread::sleep_for(std::chrono::seconds(5));
        return 5 * x;
      }};
}

## An actor that calls *takes_long_time_to_respond* actor

This is an implementation of your typical actor that would send a message
to a buddy actor. Now the buddy actor is kind of slow in responding back.

Also this actor implements more than 1 behavior.

What we are trying to learn/demonstrate here is that while SomeOther actor
is waiting for a response from buddy can we invoke other behaviors ?

In [7]:
caf::behavior some_other_actor(caf::event_based_actor *self,
                               const caf::actor &buddy) {

  return {

      // behaviors of our actor

      // this is a behavior 1 that simply receives x and multiply it with 2
      [=](int x) -> int {
        caf::aout(self) << "In Behavior 1 " << std::endl;
        return 2 * x;
      },

      // this is a behavior 2 that simply receives x and returns it with some
      // prefix
      [=](const std::string &x) -> std::string {
        caf::aout(self) << "In Behavior 2 " << std::endl;
        return "Hey " + x;
      },

      // this is a behavior 3 that sends message to buddy actor and waits for
      // its response
      //
      // Here we used 'then'. This means that the mailbox of this actor will
      // be active to receive the messages for other behaviors in the mean time
      [=](caf::get_atom) {
        self->request(buddy, caf::infinite, 10).then([=](int result) {
          caf::aout(self) << "Received result from buddy " << result
                          << std::endl;
        });
      }};
}


## Main / Entrypoint

In [8]:
void caf_main() {
    
  caf::actor_system_config cfg;
  caf::actor_system system{cfg};
  caf::scoped_actor self{system};

  auto buddy = self->spawn(takes_long_time_to_respond);
  auto some_one = self->spawn(some_other_actor, buddy);

  // time to send messages
  //
  // Again what we intend to learn here is that we would issue
  // message to behavior 3 of some_one and we know that it would long
  // time. In the mean time we would send message to its other behaviors
  // and see if they respond back before we get result from behavior 3.
  //
  // Since we are using 'then' we expect that even though we sent the messages
  // to behavior 1 and 2, they would get triggered/processed even though the behavior
  // 3 has still not finished

  self->send(some_one, caf::get_atom::value); // behavior 3

  self->send(some_one, "SomeMessage"); // behavior 2

  self->send(some_one, 4); // behavior 1

  std::cout << "Press any char to terminate .." << std::endl;
  std::cin.get();
}

If you see "In behavior 2" and "In Behavior 1" then it would mean that these messages were sent and processed while we were waiting for long running behavior 3. This is the expected behavior !

Run it to verify and make sure to wait for few seconds before pressing the enter.

In [9]:
caf_main()

Press any char to terminate ..
In Behavior 2 
In Behavior 1 


 e


Received result from buddy 50
